In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

In [9]:
#set up class to calculate options implied volatility
class Implied:
    
    def __init__(self, S, K, T, r, option_market, option_type):
        
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.option_market = option_market
        self.option_type = option_type
    
    def funx(self,vol):
        
        if self.option_type == 'c':
            d1 = (1/(vol*np.sqrt(self.T)))*(np.log(self.S/self.K) + (self.r*self.T) + ((self.T*vol**2)/2))
            d2 = (1/(vol*np.sqrt(self.T)))*(np.log(self.S/self.K) + (self.r*self.T) - ((self.T*vol**2)/2))
            fc = self.S*norm.cdf(d1) - (self.K*np.exp(-(self.r)*self.T)*norm.cdf(d2)) - self.option_market
            return fc
        
        if self.option_type == 'p':
            d1 = (1/(vol*np.sqrt(self.T)))*(np.log(self.S/K) + (self.r*self.T) + ((self.T*vol**2)/2))
            d2 = (1/(vol*np.sqrt(self.T)))*(np.log(self.S/K) + (self.r*self.T) - ((self.T*vol**2)/2))
            fc = -self.S*norm.cdf(-d1) + (self.K*np.exp(-(self.r)*self.T)*norm.cdf(-d2)) - self.option_market
            return fc
    
        else:
            
            print('Define option_type as "c" or "p"')
            
        
    def derivative_fx(self,vol):
        d1 = (1/(vol*np.sqrt(self.T)))*(np.log(self.S/self.K) + (self.r*self.T) + ((self.T*vol**2)/2))
        vega = self.S*np.sqrt(self.T)*(1/np.sqrt(2*np.pi))*np.exp(-0.5*d1**2)
        return vega
    
    def pricing(self, implied_vol):
        
        if self.option_type == 'c':
            d1 = (1/(implied_vol*np.sqrt(self.T)))*(np.log(self.S/self.K) + (self.r*self.T) + ((self.T*implied_vol**2)/2))
            d2 = (1/(implied_vol*np.sqrt(self.T)))*(np.log(self.S/self.K) + (self.r*self.T) - ((self.T*implied_vol**2)/2))
            fc = self.S*norm.cdf(d1) - (self.K*np.exp(-(self.r)*self.T)*norm.cdf(d2))
            return fc
        
        if self.option_type == 'p':
            d1 = (1/(implied_vol*np.sqrt(self.T)))*(np.log(self.S/K) + (self.r*self.T) + ((self.T*implied_vol**2)/2))
            d2 = (1/(implied_vol*np.sqrt(self.T)))*(np.log(self.S/K) + (self.r*self.T) - ((self.T*implied_vol**2)/2))
            fp = -self.S*norm.cdf(-d1) + (self.K*np.exp(-(self.r)*self.T)*norm.cdf(-d2))
            return fp
    
    def drive_newton_raphson(self, accuracy, vol0):
        
        vol_imp = vol0 - self.funx(vol = vol0)/self.derivative_fx(vol = vol0)
        approx = self.pricing(implied_vol = vol_imp)
        error = abs(self.option_market - approx)
        
        while error > accuracy:
            
            vol_imp = vol_imp - self.funx(vol = vol_imp)/self.derivative_fx(vol = vol_imp)
            approx = self.pricing(implied_vol = vol_imp)
            error = abs(self.option_market - approx)
        
    
        return vol_imp
        
        



In [10]:
#option parameters for pricing method; C is the "true" value of the option:
S0 = 36.12; K = 35; T = 7/52; r = 0.07; C = 2.15; vol0 = (2*np.sqrt(2*np.pi)*C)/S0


In [11]:
#option type c -> call option
Implied = Implied(S = S0, K = K, T = T, r = r, option_market = C, option_type = 'c')
vol_imp = vol0 - Implied.funx(vol = vol0)/Implied.derivative_fx(vol = vol0)
print('First Guess:', vol_imp)



0.25155749576554653

In [12]:
#Iterate until desired precision level is reached:

level = .0000000001#Precission Decimals
approx = Implied.pricing(implied_vol = vol_imp)
error = abs(C - approx)

arr = np.array([C,approx,vol_imp,error]); vals = len(arr)

#iterate:
while error > level:
    
    vol_imp = vol_imp - Implied.funx(vol = vol_imp)/Implied.derivative_fx(vol = vol_imp)
    approx = Implied.pricing(implied_vol = vol_imp)
    error = abs(C - approx)
    add = np.array([C,approx,vol_imp,error])
    arr = np.concatenate((arr,add))  
        
arr = np.reshape(arr, (int(len(arr)/vals), vals))  

#insert starting guess
add = np.array([C,Implied.pricing(implied_vol = vol0),vol0,abs(C - Implied.pricing(implied_vol = vol0))])
add = np.reshape(add, (int(len(add)/vals), vals))
arr = np.concatenate((add,arr))
     
results = pd.DataFrame(arr)
names = ['Price','Approximated_Price','Implied_Vol','error']
results.columns = names
results

,Price,Approximated_Price,Implied_Vol,error
0,2.15,2.375969,0.298408,2.259689e-01
1,2.15,2.152864,0.251557,2.863601e-03
2,2.15,2.150001,0.250947,6.773273e-07
3,2.15,2.150000,0.250947,3.774758e-14
